<a href="https://colab.research.google.com/github/surasakcho/A_A/blob/master/01_05_colab_ls8_extract_all_pixels_by_scene_id_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libraries & Mount google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install rasterio
!pip install geopandas


In [4]:
#Step 1
!apt-get update
#Step 2
!apt-get install libgdal-dev -y
#Step 3
!apt-get install python-gdal -y
#Step 4
!apt-get install python-numpy python-scipy -y
#Step 5


Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgdal

In [0]:
import numpy as np
import pandas as pd
import os, shutil
from glob import glob
import re
import datetime
from datetime import datetime
from tqdm import tqdm_notebook
import pickle
import matplotlib.pyplot as plt
import geopandas as gpd
from pyproj import CRS
import rasterio
from rasterio import plot
from rasterio.transform import from_origin
from rasterio.warp import reproject, Resampling
from rasterio.mask import mask
from shapely.geometry import Polygon, mapping
import gdal
%matplotlib inline

# Define paths

In [0]:

my_drive_path = "/content/drive/My Drive/"

landsat_path = os.path.join(my_drive_path, '!landsat8-prep')
ndvi_path = os.path.join(landsat_path, 'ndvi')
vrt_path = os.path.join(landsat_path, 'vrt')
polygon_prov_path = os.path.join(landsat_path, 'plot_polygon_prov_dissolved')
pixel_rowcol_map_path = os.path.join(landsat_path, 'pixel_rowcol_mapping')
pixel_rowcol_map_by_scene_id_path = os.path.join(landsat_path, 'pixel_rowcol_map_by_scene_id')
temp_path = os.path.join(landsat_path, 'temp')
pixel_values_raw_path = os.path.join(landsat_path, 'pixel_values_raw')


# Define functions

In [0]:

def list_files_re(rootpath, filename_re=None, folder_re=None ):
    '''
    rootpath : root path to lookup files
    filename_re : regular expression to search for filename
    folder_re : regular expression to search for folder

    return : a list of filepaths
    '''


    list_files = []
    for folder, _, files in os.walk(rootpath):
        for file in files:     
            if filename_re == None:
                filename_re = '.*'
            if folder_re == None:
                folder_re = '.*'
                
            if ((re.search(filename_re, file) != None) & (re.search(folder_re, folder) != None)):
                list_files.append(os.path.join(folder, file))
        
    return list_files    
        
        
    

# Extract pixel values

In [0]:
list_pixel_rowcol_map = list_files_re(pixel_rowcol_map_by_scene_id_path, 'df_pixel_rowcol_map_\\d{6}.parquet')


In [65]:
import random
random.shuffle(list_pixel_rowcol_map)
list_pixel_rowcol_map

['/content/drive/My Drive/!landsat8-prep/pixel_rowcol_map_by_scene_id/df_pixel_rowcol_map_130054.parquet',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_map_by_scene_id/df_pixel_rowcol_map_127050.parquet',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_map_by_scene_id/df_pixel_rowcol_map_128056.parquet',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_map_by_scene_id/df_pixel_rowcol_map_128049.parquet',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_map_by_scene_id/df_pixel_rowcol_map_129049.parquet',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_map_by_scene_id/df_pixel_rowcol_map_126050.parquet',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_map_by_scene_id/df_pixel_rowcol_map_130055.parquet',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_map_by_scene_id/df_pixel_rowcol_map_127055.parquet',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_map_by_scene_id/df_pixel_rowcol_map_129054.parquet',
 '/content/drive/My Drive/!landsat8-p

In [0]:
for pixel_map in tqdm_notebook(list_pixel_rowcol_map, 'Iterate over scene_id'):
    scene_id = pixel_map.split('.')[0][-6:]
    print(scene_id)

    vrt_filepath = list_files_re(vrt_path, f'stacked_ls8_{scene_id}.vrt')[0]

    df_map = pd.read_parquet(pixel_map)  
    
    
    rows = list(df_map['row'].unique())
    
    
    ds = gdal.Open(vrt_filepath)
    bands = ds.RasterCount

    #get band info
    list_band_desc = []
    for band_id in range(1, bands + 1):

        list_band_desc.append((band_id, ds.GetRasterBand(band_id).GetDescription()))

    df_band_info = pd.DataFrame(list_band_desc, columns=['band_id', 'band_desc'])
    df_band_info['date'] = df_band_info['band_desc'].str.slice(-8)
    df_band_info['year'] = df_band_info['band_desc'].str.slice(-8,-4)

    for grp in tqdm_notebook(df_band_info.groupby('year'), 'Iterate over year'):
        df_pixel_values = df_map.copy().reset_index(drop=True)

        year = grp[0]
        out_path = os.path.join(pixel_values_raw_path, f'df_pixel_values_{scene_id}_{year}.parquet')

        if os.path.exists(out_path):
            continue

        print(f'Processing : {year}')
        df_band_year = grp[1]    
        for band_id in tqdm_notebook(list(df_band_year['band_id']), 'Iterate over band (date)'):
            band_desc = ds.GetRasterBand(band_id).GetDescription()
            rst_values = ds.GetRasterBand(band_id).ReadAsArray()
            
            for sub_rows in np.array_split(rows, 10):
                df_pixel_values_curr = df_pixel_values.copy()[df_pixel_values['row'].isin(sub_rows)]
                df_pixel_values.at[df_pixel_values_curr.index, band_desc] = rst_values[df_pixel_values_curr['row'], df_pixel_values_curr['col']]

        if ~os.path.exists(out_path):
            df_pixel_values.to_parquet(out_path)
            print(f'{out_path} created.')
        
            

130054


Processing : 2014


,band_id,band_desc
0,1,ls8_126049_20130408
1,2,ls8_126049_20130415
2,3,ls8_126049_20130501
3,4,ls8_126049_20130517
4,5,ls8_126049_20130602
...,...,...
136,137,ls8_126049_20191228
137,138,ls8_126049_20200113
138,139,ls8_126049_20200129
139,140,ls8_126049_20200214


,band_id,band_desc,date,year
0,1,ls8_126049_20130408,20130408,2013
1,2,ls8_126049_20130415,20130415,2013
2,3,ls8_126049_20130501,20130501,2013
3,4,ls8_126049_20130517,20130517,2013
4,5,ls8_126049_20130602,20130602,2013
...,...,...,...,...
136,137,ls8_126049_20191228,20191228,2019
137,138,ls8_126049_20200113,20200113,2020
138,139,ls8_126049_20200129,20200129,2020
139,140,ls8_126049_20200214,20200214,2020


In [16]:
df_pixel_values

,row,col,scene_id,ls8_126049_20130408
0,2452,3015,126049,NaN
1,2453,3014,126049,NaN
2,2453,3015,126049,NaN
3,2453,3016,126049,NaN
4,2453,3017,126049,NaN
...,...,...,...,...
10454503,7740,4755,126049,NaN
10454504,7740,4756,126049,NaN
10454505,7740,4757,126049,NaN
10454506,7740,4758,126049,NaN


# Test

In [32]:
for pixel_map in tqdm_notebook(list_pixel_rowcol_map):
    scene_id = pixel_map.split('.')[0][-6:]
    print(scene_id)

    vrt_filepath = list_files_re(vrt_path, f'stacked_ls8_{scene_id}.vrt')[0]

    df_pixel_values = pd.read_parquet(pixel_map)  
    df_pixel_values = df_pixel_values.reset_index(drop=True)
    
    rows = list(df_pixel_values['row'].unique())
    
    
    ds = gdal.Open(vrt_filepath)
    bands = ds.RasterCount

    for band_id in tqdm_notebook(range(1, bands + 1)):
        band_desc = ds.GetRasterBand(band_id).GetDescription()
        rst_values = ds.GetRasterBand(band_id).ReadAsArray()
        #height, width = rst_values.shape
        
        for sub_rows in tqdm_notebook(np.array_split(rows, 10)):
            #print(sub_rows)
            df_pixel_values_curr = df_pixel_values.copy()[df_pixel_values['row'].isin(sub_rows)]
            df_pixel_values.at[df_pixel_values_curr.index, band_desc] = rst_values[df_pixel_values_curr['row'], df_pixel_values_curr['col']]



        

126049


KeyboardInterrupt: ignored

In [0]:
df_pixel_values.to_parquet(os.path.join(temp_path, 'df_pixel_values.parquet'))

In [37]:
df_pixel_values

,row,col,scene_id,ls8_126049_20130408,ls8_126049_20130415,ls8_126049_20130501,ls8_126049_20130517,ls8_126049_20130602
0,2452,3015,126049,0.083340,0.538726,0.604012,0.268359,0.103543
1,2453,3014,126049,0.081738,0.340343,0.439388,0.318555,0.101545
2,2453,3015,126049,0.082562,0.412698,0.522412,0.300169,0.101157
3,2453,3016,126049,0.082701,0.460278,0.538818,0.244538,0.101501
4,2453,3017,126049,0.083927,0.482560,0.573487,0.342381,0.100789
...,...,...,...,...,...,...,...,...
10454503,7740,4755,126049,0.000000,0.000000,0.000000,0.000000,0.000000
10454504,7740,4756,126049,0.000000,0.000000,0.000000,0.000000,0.000000
10454505,7740,4757,126049,0.000000,0.000000,0.000000,0.000000,0.000000
10454506,7740,4758,126049,0.000000,0.000000,0.000000,0.000000,0.000000


In [0]:
band_id = 2
band_desc = ds.GetRasterBand(band_id).GetDescription()
rst_values = ds.GetRasterBand(band_id).ReadAsArray()

In [39]:
band_desc

'ls8_126049_20130415'

In [41]:
rst_values[(2453, 3017)]

0.4825599292678511